In [41]:
import pandas as pd
import numpy as np
from typing import Tuple, Dict, Any
import re


class Config:
    """Configuration constants for VAT and duty calculations."""

    VAT_RATES = {
        'DE': 0.19, 'PT': 0.23, 'ES': 0.21, 'IE': 0.23,
        'SE': 0.25, 'NL': 0.21, 'DK': 0.25, 'FI': 0.255,
        'IT': 0.22, 'AT': 0.20, 'BE': 0.21, 'EE': 0.22
    }

    NL_VAT_RATE = 0.21
    DEFAULT_COMMISSION_RATE = 0.2
    IE_COMMISSION_RATE = 0.3
    CONSIGNMENT_THRESHOLD = 150

    # Output directory for CSV reports
    OUTPUT_DIR = "./reports/"

    low_value_columns = ['MRN', 'Line Item Quantity Imported', 'Line Item Quantity Returned', 'Line Item Unit Price',
                         'Consignment Value', 'VAT Rate', 'Consignee Country']

    high_value_columns = ['MRN', 'HS CODE', 'Line Item Quantity Imported', 'Line Item Quantity Returned',
                          'Line Item Unit Price',
                          'Consignment Value', 'VAT Rate', 'Consignee Country']


class DutyProcessor:

    @staticmethod
    def process_duty_data(df: pd.DataFrame) -> Dict[str, float]:
        # Extract first 4 digits from Goods code
        df['Goods_Code_4'] = df['Goods code'].astype(str).str[:4]

        df = df[df['Origin'] == 'ERGA OMNES']

        # Create new column with parsed duty rate
        df['Duty_rate'] = df['Duty'].apply(DutyProcessor.parse_duty_rate)

        # Summary of parsing
        total_rows = len(df)
        parsed_count = df['Duty_rate'].notna().sum()
        unparsed_count = total_rows - parsed_count
        print(f"Duty parsing: {parsed_count}/{total_rows} parsed, {unparsed_count} unparsed (set as NaN).")

        # Pick the biggest (max) duty rate per 4-digit goods code
        max_duty = df.groupby('Goods_Code_4')['Duty_rate'].max()

        # Remove codes without a numeric duty rate (NaN values)
        max_duty = max_duty.dropna()

        # Convert to dictionary and sort by duty rate (optional, for logging)
        duty_dict = max_duty.to_dict()

        return duty_dict

    @staticmethod
    def parse_duty_rate(val):
        """
        Parse duty strings and return duty rate as float (e.g. '12.000 %' -> 0.12).
        Returns np.nan when no percentage can be found (e.g. 'NAR', 'Cond: ...').
        """
        if pd.isna(val):
            return np.nan
        s = str(val).strip()
        # Look for percentage like '12.000 %' or '12%' with optional spaces and commas
        m = re.search(r'([\d]+[.,]?\d*)\s*%', s)
        if m:
            num = m.group(1).replace(',', '.')
            try:
                return float(num) / 100.0
            except ValueError:
                return np.nan
        # Fallback: if string is purely numeric (no percent sign) take it as percent
        m2 = re.search(r'^([\d]+[.,]?\d*)$', s)
        if m2:
            num = m2.group(1).replace(',', '.')
            try:
                return float(num) / 100.0
            except ValueError:
                return np.nan
        # No quantitative value found
        return np.nan


class DataLayer:

    @staticmethod
    def load_data(csv_path: str) -> pd.DataFrame:
        df = pd.read_csv(csv_path)

        df = DataLayer.clean_data(df)  # clean data
        df = DataLayer.add_calculated_fields(df)  # add calculated fields

        low_value_df, high_value_df = DataLayer.separate_data(df,
                                                              Config.CONSIGNMENT_THRESHOLD)  # separate data based on consignment value to two dataframes

        return low_value_df, high_value_df

    @staticmethod
    def clean_data(df: pd.DataFrame) -> pd.DataFrame:
        df = df[~df['Consignee Country'].isin(['IC', 'CH'])]  # Exclude IC and CH countries
        df['MRN'] = df['MRN'].replace(['#N/A', 'N/A', 'NA', 'na', ''], pd.NA)  # standardize missing values
        df.loc[df['MRN'].isna(), 'MRN'] = df.loc[df['MRN'].isna(), 'Parcel ID']  # copy Parcel ID where MRN is missing
        return df

    @staticmethod
    def add_calculated_fields(
            df: pd.DataFrame) -> pd.DataFrame:  # we add Cosnignment Value ( sum of all parcels under the same MRN ) and VAT Rate
        # Calculate line item total value (quantity × unit price)
        df['Line Item Total Value'] = df['Line Item Quantity Imported'] * df['Line Item Unit Price']

        # Consignment Value: sum of all line item total values per MRN
        df['Consignment Value'] = df.groupby('MRN')['Line Item Total Value'].transform('sum')

        df['VAT Rate'] = df['Consignee Country'].map(Config.VAT_RATES)

        if df['VAT Rate'].isna().any():
            missing_countries = df[df['VAT Rate'].isna()]['Consignee Country'].unique()
            print(f"⚠️  WARNING: Missing VAT rates for countries: {missing_countries}")

        return df

    @staticmethod
    def separate_data(df: pd.DataFrame, threshold: int) -> Tuple[pd.DataFrame, pd.DataFrame]:
        high_value_df = df[df['Consignment Value'] > threshold].copy()
        low_value_df = df[df['Consignment Value'] <= threshold].copy()
        return low_value_df, high_value_df


class Services:

    @staticmethod
    def store_lv_data(lv_consignments, vat_per_country, return_vat_per_country, DR_revenue_table) -> None:
        lv_consignments.to_csv("lv_consignments_data.csv", index=False)
        vat_per_country.to_csv("lv_vat_per_country_summary.csv", index=False)
        return_vat_per_country.to_csv("lv_returned_vat_per_country_summary.csv", index=False)
        DR_revenue_table.to_csv("lv_DR_revenue_summary.csv", index=False)

    @staticmethod
    def store_hv_data(hv_consignments, vat_per_country, vat_difference_table, return_vat_per_country, combined_refunds,
                      DR_revenue_table) -> None:
        hv_consignments.to_csv("hv_consignments_data.csv", index=False)
        vat_per_country.to_csv("hv_vat_per_country_summary.csv", index=False)
        vat_difference_table.to_csv("hv_vat_difference_summary.csv", index=False)
        return_vat_per_country.to_csv("hv_returned_vat_per_country_summary.csv", index=False)
        combined_refunds.to_csv("hv_duty_and_vat__returned_values_summary.csv", index=False)
        DR_revenue_table.to_csv("hv_DR_revenue_summary.csv", index=False)



    @staticmethod
    def summary_table(lv_list, hv_list):
        # Extract values from lists
        lv_dr_revenue = lv_list[0]
        lv_pc_return = lv_list[1]
        lv_total_vat_from_returns = lv_list[2]
        lv_total_import_vat = lv_list[3]

        hv_vat_return_from_nl = hv_list[0]
        hv_dr_revenue = hv_list[1]
        hv_vat_difference_txt = hv_list[2]
        hv_vat_to_pay_total = hv_list[3]

        # Calculate combined DR revenue
        total_dr_revenue = lv_dr_revenue + hv_dr_revenue

        # Create summary data
        summary_data = {
            'Category': ['Low Value', 'High Value', 'Combined'],
            'DR Revenue': [lv_dr_revenue, hv_dr_revenue, total_dr_revenue]
        }

        # Create DataFrame
        summary_df = pd.DataFrame(summary_data)

        # Add additional fields as separate rows for clarity
        additional_info = pd.DataFrame({
            'Metric': [
                'LV: PC Money to Return from Returns',
                'LV: Total VAT from Returns',
                'LV: Total import VAT Needs To Be Paid', # i
                'HV: VAT to Return from NL',
                'HV: VAT to Pay NL for distribution to other countries',
                'HV: VAT Difference Status'
            ],
            'Value': [
                f'€{lv_pc_return:.2f}',
                f'€{lv_total_vat_from_returns:.2f}',
                f'€{lv_total_import_vat:.2f}',
                f'€{hv_vat_return_from_nl:.2f}',
                f'€{hv_vat_to_pay_total:.2f}',
                hv_vat_difference_txt
            ]
        })

        # Save both tables to CSV
        summary_df.to_csv("summary_revenue_table.csv", index=False)
        additional_info.to_csv("summary_additional_info.csv", index=False)


    @staticmethod
    def calculate_vat_per_country(df: pd.DataFrame) -> pd.DataFrame:
        # First, get unique MRN records (to avoid counting same consignment multiple times)
        unique_consignments = df.drop_duplicates(subset=['MRN'])

        # Calculate VAT amount for each consignment
        unique_consignments['VAT Amount'] = unique_consignments['Consignment Value'] * unique_consignments['VAT Rate']

        # Group by Country and VAT Rate
        summary = unique_consignments.groupby(['Consignee Country', 'VAT Rate']).agg({
            'Consignment Value': 'sum',
            'VAT Amount': 'sum'
        }).reset_index()

        # Rename columns for clarity
        summary.columns = ['Country', 'VAT Rate', 'Total Consignment Value', 'Total VAT to Pay']

        return summary

    @staticmethod
    def calculate_return_vat_per_country(df: pd.DataFrame) -> pd.DataFrame:
        # Filter rows where items were returned
        returned_df = df[df['Line Item Quantity Returned'] > 0].copy()

        # Calculate total returned value for each line item
        returned_df['Returned Item Value'] = (
                returned_df['Line Item Quantity Returned'] *
                returned_df['Line Item Unit Price']
        )

        # Calculate VAT refund for each returned item
        returned_df['VAT Refund'] = returned_df['Returned Item Value'] * returned_df['VAT Rate']

        # Group by Country and VAT Rate
        summary = returned_df.groupby(['Consignee Country', 'VAT Rate']).agg({
            'Returned Item Value': 'sum',
            'VAT Refund': 'sum'
        }).reset_index()

        # Rename columns for clarity
        summary.columns = ['Country', 'VAT Rate', 'Total Returned Value', 'Total VAT Refund']

        return summary



In [42]:
class LowValueProcessor:

    @staticmethod
    def process_low_value_data(df: pd.DataFrame) -> list[Any]:
        df = LowValueProcessor.clean_columns(df)  # keep only relevant columns for low value consignments

        vat_per_country = LowValueProcessor.calculate_vat_per_country(df)  # calculate VAT per country
        return_vat_per_country = LowValueProcessor.calculate_return_vat_per_country(
            df)  # calculate VAT refunds for returned items

        # calculate DR revenue from low value returns
        DR_revenue, DR_revenue_table = LowValueProcessor.calculate_dr_revenue_from_lw_returns(return_vat_per_country)
        PC_return = return_vat_per_country['Total VAT Refund'].sum() - DR_revenue  # total PC return after DR revenue
        total_import_vat_paid = vat_per_country['Total VAT to Pay'].sum()
        total_vat_needs_to_be_collected_from_returns = return_vat_per_country['Total VAT Refund'].sum()

        # Save reports to CSV files
        Services.store_lv_data(df, vat_per_country, return_vat_per_country, DR_revenue_table)

        return [DR_revenue, PC_return, total_vat_needs_to_be_collected_from_returns, total_import_vat_paid]

    @staticmethod
    def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
        return df[Config.low_value_columns]  # keep only relevant columns for low value consignments

    @staticmethod
    def calculate_vat_per_country(df: pd.DataFrame) -> pd.DataFrame:
        return Services.calculate_vat_per_country(df)

    @staticmethod
    def calculate_return_vat_per_country(df: pd.DataFrame) -> pd.DataFrame:
        return Services.calculate_return_vat_per_country(df)

    @staticmethod
    def calculate_dr_revenue_from_lw_returns(return_vat_per_country: pd.DataFrame) -> Tuple[float, pd.DataFrame]:
        # Create a copy to avoid modifying original
        revenue_df = return_vat_per_country.copy()

        # Calculate revenue: 30% for Ireland, 20% for others
        revenue_df['Revenue'] = revenue_df.apply(
            lambda row: row['Total VAT Refund'] * 0.30 if row['Country'] == 'IE'
            else row['Total VAT Refund'] * 0.20,
            axis=1
        )

        # Keep only Country and Revenue columns
        result_df = revenue_df[['Country', 'Revenue']]

        # Calculate total revenue
        total_revenue = result_df['Revenue'].sum()

        return total_revenue, result_df




In [43]:
class HighValueProcessor:

    @staticmethod
    def process_high_value_data(df: pd.DataFrame) -> list[Any]:
        df = HighValueProcessor.clean_columns(df)  # keep only relevant columns for high value consignments

        # calculate VAT to return from NL
        vat_to_return_from_nl = HighValueProcessor.calculate_vat_to_return_from_nl(df)

        vat_per_country = HighValueProcessor.calculate_vat_per_country(df) # calculate VAT per country to submit to NL
        vat_to_pay_total = vat_per_country['Total VAT to Pay'].sum()

        # create VAT difference table ( if PC pays DR or DR pays PC for vat differences)
        vat_difference_table = HighValueProcessor.create_vat_difference_table(vat_per_country)
        vat_difference_payment_txt = HighValueProcessor.calculate_vat_difference_payment_txt(vat_difference_table)

        # calculate VAT refunds for returned items
        return_vat_per_country = LowValueProcessor.calculate_return_vat_per_country(df)

        # getting duty that should be returned for returned items
        return_duty_amount, duty_returned_by_country = HighValueProcessor.calculate_duty_for_returned_items(
            df, DutyProcessor.process_duty_data(pd.read_excel('Duties Import Jan 99.xlsx'))
        )

        # Merge duty and VAT refunds by country
        combined_refunds = HighValueProcessor.duty_vat_hv_merge(return_vat_per_country, duty_returned_by_country)

        #calculate DR revenue from high value returns
        DR_revenue_hv, DR_revenue_table = HighValueProcessor.calculate_dr_revenue_from_hw_returns(combined_refunds)

        # Save reports to CSV files
        Services.store_hv_data(df, vat_per_country, vat_difference_table, return_vat_per_country, combined_refunds,
                               DR_revenue_table)

        return [vat_to_return_from_nl, DR_revenue_hv, vat_difference_payment_txt, vat_to_pay_total]

    @staticmethod
    def calculate_vat_difference_payment_txt(df: pd.DataFrame, ) -> str:
        vat_in_nl = df['VAT Paid In NL'].sum()
        vat_in_consignee_country = df['VAT Paid In Consignee Country'].sum()
        difference = vat_in_nl - vat_in_consignee_country

        # if in NL we paied more VAT than in consignee country, we got some spare money left , hence we need to return it to PC
        if difference > 0:
            return f'DR needs to return PC an amount of €{difference:.4f} for VAT differences.'
        return f'PC needs to pay DR extra an amount of €{abs(difference):.4f} for VAT differences.'

    @staticmethod
    def create_vat_difference_table(vat_per_country: pd.DataFrame) -> pd.DataFrame:
        vat_difference_table = vat_per_country.copy()

        # Calculate VAT that was paid in NL (21% of consignment value)
        vat_difference_table['VAT Paid In NL'] = vat_difference_table['Total Consignment Value'] * Config.NL_VAT_RATE

        # Rename 'Total VAT to Pay' to 'VAT Paid In Consignee Country' for clarity
        vat_difference_table = vat_difference_table.rename(columns={
            'Total VAT to Pay': 'VAT Paid In Consignee Country'
        })

        # Reorder columns
        vat_difference_table = vat_difference_table[
            ['Country', 'VAT Rate', 'Total Consignment Value', 'VAT Paid In NL', 'VAT Paid In Consignee Country']]

        return vat_difference_table

    @staticmethod
    def calculate_dr_revenue_from_hw_returns(combined_refunds: pd.DataFrame) -> Tuple[float, pd.DataFrame]:
        # Create a copy to avoid modifying original
        revenue_df = combined_refunds.copy()

        # Calculate revenue: 30% for Ireland, 20% for others
        revenue_df['Revenue'] = revenue_df.apply(
            lambda row: row['Total Refund'] * 0.30 if row['Country'] == 'IE'
            else row['Total Refund'] * 0.20,
            axis=1
        )

        # Keep only Country and Revenue columns
        result_df = revenue_df[['Country', 'Revenue']]

        # Calculate total revenue
        total_revenue = result_df['Revenue'].sum()

        return total_revenue, result_df

    @staticmethod
    def duty_vat_hv_merge(vat_df: pd.DataFrame, duty_df: pd.DataFrame) -> pd.DataFrame:
        # Merge VAT and Duty dataframes on Country
        merged_df = pd.merge(
            vat_df,
            duty_df[['Country', 'Total Duty Returned']],
            on='Country',
            how='outer'
        )

        # Fill all NaN values with 0
        merged_df = merged_df.fillna({
            'VAT Rate': 0,
            'Total Returned Value': 0,
            'Total VAT Refund': 0,
            'Total Duty Returned': 0
        })

        # Calculate Total Refund (VAT + Duty)
        merged_df['Total Refund'] = merged_df['Total VAT Refund'] + merged_df['Total Duty Returned']

        # Reorder columns for clarity
        merged_df = merged_df[[
            'Country',
            'VAT Rate',
            'Total Returned Value',
            'Total VAT Refund',
            'Total Duty Returned',
            'Total Refund'
        ]]

        return merged_df

    @staticmethod
    def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
        return df[Config.high_value_columns]  # keep only relevant columns for low value consignments

    @staticmethod
    def calculate_vat_per_country(df: pd.DataFrame) -> pd.DataFrame:
        return Services.calculate_vat_per_country(df)

    @staticmethod
    def calculate_return_vat_per_country(df: pd.DataFrame) -> pd.DataFrame:
        return Services.calculate_return_vat_per_country(df)

    @staticmethod
    def calculate_duty_for_returned_items(df: pd.DataFrame, duty_dict: Dict[str, float]) -> Tuple[float, pd.DataFrame]:
        # Filter rows where items were returned
        returned_df = df[df['Line Item Quantity Returned'] > 0].copy()

        # Extract first 4 digits from HS CODE
        returned_df['Goods_Code_4'] = returned_df['HS CODE'].astype(str).str[:4]

        # Map duty rates using the provided duty dictionary
        returned_df['Duty Rate'] = returned_df['Goods_Code_4'].map(duty_dict)

        # Calculate total returned value for each line item
        returned_df['Returned Item Value'] = (
                returned_df['Line Item Quantity Returned'] *
                returned_df['Line Item Unit Price']
        )

        # Calculate Duty for each returned item
        returned_df['Duty Amount'] = returned_df['Returned Item Value'] * returned_df['Duty Rate']

        # Group by country and sum duty amounts
        duty_by_country = returned_df.groupby('Consignee Country').agg({
            'Returned Item Value': 'sum',
            'Duty Amount': 'sum'
        }).reset_index()

        # Rename columns for clarity
        duty_by_country.columns = ['Country', 'Total Returned Value', 'Total Duty Returned']

        # Calculate total duty across all countries
        total_duty = duty_by_country['Total Duty Returned'].sum()

        return total_duty, duty_by_country

    @staticmethod
    def calculate_vat_to_return_from_nl(df: pd.DataFrame) -> float:
        # First, get unique MRN records (to avoid counting same consignment multiple times)
        unique_consignments = df.drop_duplicates(subset=['MRN'])

        # Calculate NL VAT amount for each consignment
        unique_consignments['VAT Amount'] = unique_consignments['Consignment Value'] * Config.NL_VAT_RATE

        # sum all the values
        total_nl_vat = unique_consignments['VAT Amount'].sum()

        return total_nl_vat

In [44]:
def main():
    csv_path = "TED BAKER DUTY CLAIM BACK Jul-Sep v2.csv"

    print("📊 Starting VAT Analysis...")

    low_value_df, high_value_df = DataLayer.load_data(csv_path)

    low_values_data = LowValueProcessor.process_low_value_data(
        low_value_df)  # [DR revenue from return from LV, PC's money that must be returned from returns from LV, Total VAT That Needs to be Collected From Returns, Total Import VAT Paid]
    high_values_data = HighValueProcessor.process_high_value_data(
        high_value_df)  # [VAT that needs to be returned from NL, DR revenue from return from HV, VAT difference payment text, vat to pay total]

    # Summary Output
    Services.summary_table(low_values_data, high_values_data)

    print("\n✅ Analysis complete!")

In [45]:
if __name__ == "__main__":
    main()

📊 Starting VAT Analysis...


/var/folders/yz/6pjrzx695kzdch62s9tfzthh0000gn/T/ipykernel_53422/1804873111.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MRN'] = df['MRN'].replace(['#N/A', 'N/A', 'NA', 'na', ''], pd.NA)  # standardize missing values
/var/folders/yz/6pjrzx695kzdch62s9tfzthh0000gn/T/ipykernel_53422/1804873111.py:212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_consignments['VAT Amount'] = unique_consignments['Consignment Value'] * unique_consignments['VAT Rate']
/var/folders/yz/6pjrzx695kzdch62s9tfzthh0

Duty parsing: 14189/22856 parsed, 8667 unparsed (set as NaN).

✅ Analysis complete!
